# Module 5: Model Evaluation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NabKh/ML-for-Materials-Science/blob/main/Tutorial-07-ML-Discovery/notebooks/05_model_evaluation.ipynb)

> **Before You Start:** Please check the [INSTALLATION_GUIDE.md](../../INSTALLATION_GUIDE.md) for setup instructions. For Google Colab:
> ```python
> !pip install pymatgen matminer shap -q
> ```
> Then restart the runtime (Runtime → Restart runtime).

---

## 🎯 Learning Objectives

1. **Use** proper metrics (MAE, RMSE, R²) and understand when each matters
2. **Implement** k-fold cross-validation for robust evaluation
3. **Visualize** learning curves to diagnose overfitting/underfitting
4. **Tune** hyperparameters with Optuna

---

**⏱️ Estimated time: 75 minutes** | **📚 Difficulty: 🟡 Intermediate**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Create figures directory
os.makedirs('figures', exist_ok=True)

from sklearn.model_selection import (
    train_test_split, cross_val_score, KFold, learning_curve
)
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

try:
    import optuna
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    HAS_OPTUNA = True
except:
    HAS_OPTUNA = False
    print("Optuna not installed, hyperparameter tuning section will use grid search")

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
print("✅ Ready!")

<cell_type>markdown</cell_type>## 1. Understanding Evaluation Metrics

### 📖 Theory

<div style="background: linear-gradient(135deg, #1e293b 0%, #0f172a 100%); padding: 20px; border-radius: 10px; border-left: 4px solid #6366f1;">

**Why proper evaluation matters:**
- Training error is always optimistic (model has "seen" the data)
- We need to estimate how well the model generalizes to **unseen data**
- Different metrics capture different aspects of error

</div>

### Regression Metrics

**Mean Absolute Error (MAE):**
$$\text{MAE} = \frac{1}{n}\sum_{i=1}^{n}|y_i - \hat{y}_i|$$

- Measures average magnitude of errors
- Same units as the target variable
- **Robust to outliers** (linear penalty)

**Root Mean Squared Error (RMSE):**
$$\text{RMSE} = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}$$

- Penalizes large errors more heavily (quadratic penalty)
- Same units as target variable
- **Sensitive to outliers**

**Coefficient of Determination (R²):**
$$R^2 = 1 - \frac{SS_{res}}{SS_{tot}} = 1 - \frac{\sum(y_i - \hat{y}_i)^2}{\sum(y_i - \bar{y})^2}$$

- Proportion of variance explained by the model
- Ranges from $-\infty$ to 1 (1 = perfect, 0 = mean baseline)
- **Unitless** - good for comparing models

### When to Use Each Metric

| Metric | Best For | Limitation |
|--------|----------|------------|
| MAE | Interpretable errors, outlier-robust | Doesn't penalize large errors |
| RMSE | When large errors are especially bad | Sensitive to outliers |
| R² | Comparing models, variance explained | Can be misleading for non-linear patterns |

In [ ]:
# Create sample data
np.random.seed(42)
n = 500
X = np.random.randn(n, 10)
y = 2*X[:, 0] + 1.5*X[:, 1]**2 + 0.5*np.random.randn(n)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# Train model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_s, y_train)
y_pred = rf.predict(X_test_s)

# Calculate metrics
print("Model Performance Metrics:")
print(f"  MAE:  {mean_absolute_error(y_test, y_pred):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"  R²:   {r2_score(y_test, y_pred):.4f}")

<cell_type>markdown</cell_type>## 2. Cross-Validation

### 📖 Theory

<div style="background: rgba(14, 165, 233, 0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #0ea5e9;">

**The Problem with a Single Train-Test Split:**
- Results depend heavily on which samples end up in train vs test
- Single estimate has high variance
- May get "lucky" or "unlucky" with the split

**Solution: K-Fold Cross-Validation**

1. Split data into $k$ equal folds
2. For each fold $i$: train on folds $\{1,...,k\} \setminus i$, test on fold $i$
3. Average the $k$ test scores

</div>

### Mathematical Framework

The cross-validated score estimates the expected prediction error:

$$\text{CV}(k) = \frac{1}{k}\sum_{i=1}^{k}\text{Error}(\text{fold}_i)$$

**Variance of CV estimate decreases with more folds:**
$$\text{Var}(\text{CV}) \approx \frac{\sigma^2}{k}$$

But more folds = more computation and potential for correlated training sets.

**Common choices:**
- **k=5**: Good balance (80% train, 20% test each fold)
- **k=10**: Lower variance, more computation
- **LOOCV (k=n)**: Unbiased but high variance, expensive

In [ ]:
# 5-fold cross-validation with professional visualization
rf = RandomForestRegressor(n_estimators=100, random_state=42)

cv_scores = cross_val_score(rf, X, y, cv=5, scoring='r2')

print("5-Fold Cross-Validation Results:")
print(f"  Scores: {cv_scores.round(4)}")
print(f"  Mean:   {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# Professional visualization
fig, ax = plt.subplots(figsize=(10, 5), facecolor='white')

colors = {
    'bars': '#6366f1',
    'mean': '#ef4444',
    'text': '#1e293b',
    'grid': '#e2e8f0'
}

# Create bars
bars = ax.bar(range(1, 6), cv_scores, color=colors['bars'], alpha=0.8, 
              edgecolor='white', linewidth=2)

# Add value labels on bars
for bar, score in zip(bars, cv_scores):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
            f'{score:.3f}', ha='center', va='bottom', fontsize=11, 
            fontweight='bold', color=colors['text'])

# Mean and std band
ax.axhline(cv_scores.mean(), color=colors['mean'], linestyle='--', linewidth=2, 
           label=f'Mean: {cv_scores.mean():.3f}')
ax.fill_between([0.5, 5.5], cv_scores.mean()-cv_scores.std(), cv_scores.mean()+cv_scores.std(), 
                alpha=0.15, color=colors['mean'], label=f'±1 std: {cv_scores.std():.3f}')

ax.set_xlabel('Fold', fontsize=12, color=colors['text'])
ax.set_ylabel('R² Score', fontsize=12, color=colors['text'])
ax.set_title('5-Fold Cross-Validation Results', fontsize=14, fontweight='bold', color=colors['text'])
ax.set_xticks(range(1, 6))
ax.set_ylim(0, 1.05)
ax.set_xlim(0.4, 5.6)
ax.legend(loc='lower right', fontsize=10, framealpha=0.95)
ax.set_facecolor('white')
ax.grid(True, axis='y', alpha=0.3, color=colors['grid'])

# Add interpretation
ax.text(0.02, 0.02, f'Low variance (±{cv_scores.std():.3f}) indicates stable model performance', 
        transform=ax.transAxes, fontsize=10, color=colors['text'], style='italic',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='#f1f5f9', edgecolor=colors['grid']))

plt.tight_layout()
plt.savefig('figures/05_cross_validation.png', dpi=200, bbox_inches='tight', facecolor='white')
plt.show()

print("\nFigure saved to figures/05_cross_validation.png")

<cell_type>markdown</cell_type>## 3. Learning Curves: Diagnosing Model Behavior

### 📖 Theory

<div style="background: rgba(16, 185, 129, 0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #10b981;">

**Learning curves** show how model performance changes with training set size.

They help diagnose the **bias-variance tradeoff**:

$$\text{Expected Error} = \text{Bias}^2 + \text{Variance} + \text{Irreducible Noise}$$

</div>

### Interpreting Learning Curves

| Pattern | Training Score | Validation Score | Diagnosis | Solution |
|---------|---------------|------------------|-----------|----------|
| **Underfitting** | Low | Low | High bias | More complex model |
| **Overfitting** | High | Low (big gap) | High variance | More data, regularization |
| **Good fit** | High | High (converging) | Balanced | Keep current approach |

### Visual Guide

```
     Underfitting          Overfitting           Good Fit
    ┌────────────┐      ┌────────────┐      ┌────────────┐
 R² │━━━ Train   │   R² │━━━━━━━ Tr  │   R² │━━━━━━━ Tr  │
    │━━━ Valid   │      │            │      │━━━━━━ Val  │
    │            │      │━━━ Valid   │      │            │
    └────────────┘      └────────────┘      └────────────┘
       Training Size       Training Size       Training Size
```

In [ ]:
# Generate and visualize learning curves
train_sizes, train_scores, test_scores = learning_curve(
    RandomForestRegressor(n_estimators=50, random_state=42),
    X, y, cv=5, n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10),
    scoring='r2'
)

# Professional plot
fig, ax = plt.subplots(figsize=(11, 6), facecolor='white')

colors = {
    'train': '#0ea5e9',      # Sky blue
    'valid': '#ec4899',      # Pink
    'text': '#1e293b',
    'grid': '#e2e8f0'
}

train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
test_mean = test_scores.mean(axis=1)
test_std = test_scores.std(axis=1)

# Plot with confidence bands
ax.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, 
                alpha=0.2, color=colors['train'])
ax.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, 
                alpha=0.2, color=colors['valid'])
ax.plot(train_sizes, train_mean, 'o-', color=colors['train'], linewidth=2.5, 
        markersize=8, label='Training score')
ax.plot(train_sizes, test_mean, 's-', color=colors['valid'], linewidth=2.5, 
        markersize=8, label='Validation score')

ax.set_xlabel('Training Set Size', fontsize=12, color=colors['text'])
ax.set_ylabel('R² Score', fontsize=12, color=colors['text'])
ax.set_title('Learning Curve: Random Forest', fontsize=14, fontweight='bold', color=colors['text'])
ax.legend(loc='lower right', fontsize=11, framealpha=0.95)
ax.set_facecolor('white')
ax.grid(True, alpha=0.3, color=colors['grid'])
ax.set_ylim(0.5, 1.02)

# Add gap annotation
final_gap = train_mean[-1] - test_mean[-1]
ax.annotate('', xy=(train_sizes[-1]+5, test_mean[-1]), 
            xytext=(train_sizes[-1]+5, train_mean[-1]),
            arrowprops=dict(arrowstyle='<->', color='#f59e0b', lw=2))
ax.text(train_sizes[-1]+15, (train_mean[-1] + test_mean[-1])/2, 
        f'Gap: {final_gap:.2f}', fontsize=10, color='#f59e0b', fontweight='bold', va='center')

# Add interpretation box
interpretation = "Converging curves = Good fit" if final_gap < 0.1 else "Large gap = Overfitting risk"
box_color = '#10b981' if final_gap < 0.1 else '#f59e0b'
ax.text(0.02, 0.98, interpretation, transform=ax.transAxes, fontsize=11, 
        color='white', fontweight='bold', va='top',
        bbox=dict(boxstyle='round,pad=0.4', facecolor=box_color, edgecolor='none'))

plt.tight_layout()
plt.savefig('figures/05_learning_curve.png', dpi=200, bbox_inches='tight', facecolor='white')
plt.show()

print("\n💡 Interpretation:")
print(f"  - Final training score: {train_mean[-1]:.3f}")
print(f"  - Final validation score: {test_mean[-1]:.3f}")
print(f"  - Gap: {final_gap:.3f}")
if final_gap < 0.1:
    print("  ✅ Curves are converging - good model fit!")
else:
    print("  ⚠️ Large gap suggests overfitting - consider regularization or more data")
    
print("\nFigure saved to figures/05_learning_curve.png")

<cell_type>markdown</cell_type>## 4. Hyperparameter Tuning

### 📖 Theory

<div style="background: rgba(245, 158, 11, 0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #f59e0b;">

**Hyperparameters** are model settings that are NOT learned from data:
- Random Forest: `n_estimators`, `max_depth`, `min_samples_split`
- Neural Networks: learning rate, hidden layer sizes, dropout

**Goal:** Find hyperparameters that minimize validation error.

</div>

### Tuning Strategies

| Method | How it Works | Pros | Cons |
|--------|--------------|------|------|
| **Grid Search** | Try all combinations | Simple, thorough | Expensive, curse of dimensionality |
| **Random Search** | Random combinations | More efficient | May miss optimal |
| **Bayesian (Optuna)** | Learn from past trials | Most efficient | More complex |

### Optuna's Approach

Optuna uses **Tree-structured Parzen Estimator (TPE)** to model:

$$P(\text{hyperparams} | \text{good score})$$

It samples hyperparameters more likely to give good results based on previous trials.

In [ ]:
if HAS_OPTUNA:
    def objective(trial):
        n_estimators = trial.suggest_int('n_estimators', 50, 200)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
        
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            random_state=42,
            n_jobs=-1
        )
        
        scores = cross_val_score(model, X, y, cv=3, scoring='r2')
        return scores.mean()
    
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=30, show_progress_bar=True)
    
    print(f"\n✅ Best R²: {study.best_value:.4f}")
    print(f"Best parameters: {study.best_params}")
else:
    print("Optuna not available. Install with: pip install optuna")

## 📝 Exercises

### Exercise: Compare Different CV Strategies

In [ ]:
# TODO: Compare 3-fold, 5-fold, and 10-fold CV
# Which gives the lowest variance in scores?

# for k in [3, 5, 10]:
#     scores = cross_val_score(rf, X, y, cv=k, scoring='r2')
#     print(f"{k}-fold: {scores.mean():.4f} ± {scores.std():.4f}")

<cell_type>markdown</cell_type>---

## ✅ Module Summary

### Key Takeaways

<div style="background: rgba(99, 102, 241, 0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #6366f1;">

| Concept | Key Formula | When to Use |
|---------|-------------|-------------|
| **MAE** | $\frac{1}{n}\sum\|y - \hat{y}\|$ | Interpretable errors, robust to outliers |
| **RMSE** | $\sqrt{\frac{1}{n}\sum(y - \hat{y})^2}$ | When large errors are especially costly |
| **R²** | $1 - \frac{SS_{res}}{SS_{tot}}$ | Comparing models on same dataset |
| **K-Fold CV** | Average over $k$ test folds | Robust performance estimation |
| **Learning Curves** | Score vs. training size | Diagnose over/underfitting |

</div>

### Decision Flowchart

```
Evaluate Model
    │
    ├─► Calculate metrics (MAE, RMSE, R²)
    │
    ├─► Run k-fold cross-validation
    │       └─► Check variance across folds
    │
    ├─► Plot learning curves
    │       ├─► High train, low valid → Overfitting
    │       ├─► Both low → Underfitting  
    │       └─► Converging → Good fit
    │
    └─► Tune hyperparameters with Optuna
            └─► Use CV score as objective
```

### Common Pitfalls to Avoid

1. **Data leakage**: Always split BEFORE any preprocessing
2. **Over-tuning**: Use separate test set for final evaluation
3. **Ignoring variance**: Report CV mean ± std, not just mean
4. **Wrong metric**: Choose metric based on business/scientific needs

### What's Next?

In **Module 6: Explainable AI**, you'll learn to:
- Interpret model predictions with SHAP values
- Understand feature contributions
- Build trust in ML predictions

---

**📚 Continue to Module 6:** [Explainable AI](06_explainable_ai.ipynb)